# Random Forest Classifier in PySpark - Lab

## Introduction  

In this lab, we would like to a Random Forest Classifier model to study over the ecommerce behavior from a multi-category store. First, we need to download the data to your local machine, then we will load the data from the local machine onto a Pandas Dataframe.

## Objectives  

* Use the kaggle eCommerce dataset in the previous lab and reimplement using PySpark

## Instruction
* Accept the kaggle policy and download the data from here https://www.kaggle.com/code/tshephisho/ecommerce-behaviour-using-xgboost/data
* For the first model building, we'll only use the 2019-Nov csv data (which is still around ~2gb zipped)

In [1]:
# import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
pip install squarify

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import squarify
import matplotlib.dates as dates
from datetime import datetime

%matplotlib inline

In [3]:
from pyspark.sql import SparkSession #entry point for pyspark

#instantiate spark instance
spark = SparkSession.builder.appName('Random Forest eCommerce').master("local[*]").getOrCreate()

In [5]:
path = "C:\Flatiron\Test.csv" #wherever path you saved the kaggle file to
df = spark.read.csv(path, header=True, inferSchema=True)
df.printSchema() #to see the schema

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



We've already used this dataset, but feel free to explore around. Now, we want to use the pandas instead of pyspark, we have to use the `action` functions, which then means there will be a network shuffle. Earlier lab used the Iris dataset which was about ~1KB, but the current dataset may be too large, and may throw an `OutOfMemory` error if we attempt to load the data into the pandas dataframe. I would suggest only take few rows for exploratory analysis if pandas is more comfortable library. Otherwise, sticking with native PySpark functions would be much better option. 

In [6]:
 pd.DataFrame(df.take(10), columns=df.columns).transpose()

,0,1,2,3,4,5,6,7,8,9
event_time,2019-11-01 00:00:00 UTC,2019-11-01 00:00:00 UTC,2019-11-01 00:00:01 UTC,2019-11-01 00:00:01 UTC,2019-11-01 00:00:01 UTC,2019-11-01 00:00:01 UTC,2019-11-01 00:00:01 UTC,2019-11-01 00:00:02 UTC,2019-11-01 00:00:02 UTC,2019-11-01 00:00:02 UTC
event_type,view,view,view,view,view,view,view,view,view,view
product_id,1003461,5000088,17302664,3601530,1004775,1306894,1306421,15900065,12708937,1004258
category_id,2053013555631882655,2053013566100866035,2053013553853497655,2053013563810775923,2053013555631882655,2053013558920217191,2053013558920217191,2053013558190408249,2053013553559896355,2053013555631882655
category_code,electronics.smartphone,appliances.sewing_machine,None,appliances.kitchen.washer,electronics.smartphone,computers.notebook,computers.notebook,None,None,electronics.smartphone
brand,xiaomi,janome,creed,lg,xiaomi,hp,hp,rondell,michelin,apple
price,489.07,293.65,28.31,712.87,183.27,360.09,514.56,30.86,72.72,732.07
user_id,520088904,530496790,561587266,518085591,558856683,520772685,514028527,518574284,532364121,532647354
user_session,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,8e5f4f83-366c-4f70-860e-ca7417414283,755422e7-9040-477b-9bd2-6a6e8fd97387,3bfb58cd-7892-48cc-8020-2f17e6de6e7f,313628f1-68b8-460d-84f6-cec7a8796ef2,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c,df8184cc-3694-4549-8c8c-6b5171877376,5e6ef132-4d7c-4730-8c7f-85aa4082588f,0a899268-31eb-46de-898d-09b2da950b24,d2d3d2c6-631d-489e-9fb5-06f340b85be0


### Know your Customers

How many unique customers visit the site?

In [7]:
# using naitve pyspark..
from pyspark.sql.functions import countDistinct
df.select(countDistinct("user_id")).show() 

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                3696117|
+-----------------------+



In [8]:
"""About 3696117 users"""

'About 3696117 users'

Did you notice the spark progress bar when you triggered the `action` function? The `show()` function is the `action` function which means the lazy evaluation of Spark was triggered and completed a certain job. `read.csv` should have been another job. If you go to the `localhost:4040` you should be able to see 2 completed jobs under the `Jobs` tab, which are `csv` and `showString`. While a heavy job is getting executed, you can take a look at the `Executors` tab to examine the executors completing the tasks in parellel. Now, we may not see if we run this on a local machine, but this behavior should definitely be visible if you're on a cloud system, such as EMR.

### (Optional) Visitors Daily Trend

Does traffic flunctuate by date? Try using the event_time and user_id to see traffic, and draw out the plots for visualization.

In [ ]:
# try cleaning out the event_time column then using groupby/count
# import pyspark.sql.functions as F
# use this as a reference to clean the event_time column
# https://stackoverflow.com/questions/67827631/how-udf-function-works-in-pyspark-with-dates-as-arguments 

Question: We would still like to see the cart abandonment rate using the dataset. What relevant features can we use for modeling?

In [ ]:
# your answer

Now, let's build out the model.

In [ ]:
from pyspark.ml.feature import VectorAssembler

#columns you'd like to use
feature_cols = []
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)
df.show()

Is there a labeler column that we'd like to use?

In [ ]:
from pyspark.ml.feature import StringIndexer

#what should we use for the inputCol here?
labeler = StringIndexer(inputCol='', outputCol='encoded')
df = labeler.fit(df).transform(df)
df.show()

Now build the train/test dataset.

In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed=42)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol='', labelCol='encoded')
model = rf.fit(train)
predictions = model.transform(test)
# what goes in the select() function?
predictions.select().show(25)


Once the job execution is done, try evaluating on how we performed!

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

### Extra: Use the confusion matrix to see the other metrics

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

preds_and_labels = predictions.select(['prediction','encoded']).withColumn('encoded', F.col('encoded').cast(FloatType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','encoded'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())